#### import required packages and initialize the h2o application server

In [ ]:
import h2o
import pandas as pd

h2o.init(log_level="INFO")

#### load the model and the current customer dataframe

In [ ]:
m = h2o.load_model('./churn_model')
current_customers = pd.read_json('Telco_Churn_current_customers.json')
curr = h2o.H2OFrame(current_customers)

#### Score current customers

In [ ]:
# score current current customers for churn using the highest performing model
predict = m.predict(curr)

# bind the prediction to the original dataset and convert to a dataframe
curr = curr.cbind(predict)
current_customers_df = pd.DataFrame(curr.as_data_frame(), columns=curr.names)
all_columns = current_customers_df.columns
for i in all_columns:
    if i[0:12] == "Row Count - ":
        del current_customers_df[i]
print(current_customers_df.loc[:10,["customerID","predict","No","Yes"]])

current_customers.to_json('scored_customers.json')

#### shutdown the h2o application server

In [ ]:
h2o.cluster().shutdown()